In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/startup-failures/Startup Failures.csv")
data_FandI = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Finance and Insurance).csv")
data_Manu = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Manufactures).csv")
data_Retail = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Retail Trade).csv")
data_HealthC = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Health Care).csv")
data_Food = pd.read_csv("/kaggle/input/startup-failures/Startup Failure (Food and services).csv")
data_Info = pd.read_csv("/kaggle/input/startup-failures/Startup Failures (Information Sector).csv")

datas = [data,data_FandI, data_Manu,data_Retail,data_HealthC, data_Food,data_Info]
datas_n = ["data","data_FandI", "data_Manu","data_Retail","data_HealthC", "data_Food","data_Info"]

data.info()

In [ ]:
# Process to Merge the Data

i=0
for data in datas:
    print("\n")
    print(datas_n[i])
    print("\n")
    i+=1
    data.info()
    

In [ ]:
# Merging Data

dfs = [data_FandI, data_Manu, data_Retail, data_HealthC, data_Food, data_Info]
merged_data = pd.concat(dfs, ignore_index=True, sort=False)

print(merged_data.shape)

In [ ]:
merged_data.info()

In [ ]:
data_copy = merged_data.copy() # Keeping a copy of the original data for safety

# Lets Do Data Cleaning and Preprocessing

In [ ]:
merged_data.head()

In [ ]:
# Converting the Funding amounts from string/object type to float type and making a new col called Funding Amount.

merged_data[["Amount","Unit"]] = merged_data['How Much They Raised'].str.extract(r'(\d+\.?\d*)([MBK]?)')
merged_data["Amount"] = pd.to_numeric(merged_data['Amount'], errors = "coerce")
multipliers = {"M":1, "B":1000, "K":1/1000}
merged_data['Funding Amount'] = merged_data["Amount"]*merged_data["Unit"].map(multipliers).fillna(1)
merged_data['Funding Amount'] = merged_data['Funding Amount'].astype("Float64")
merged_data.drop(columns=["Amount", "Unit"], inplace = True)

merged_data.tail()

In [ ]:
billionRows = merged_data[merged_data["How Much They Raised"].str.contains("B",na=False)]
billionRows

In [ ]:
print(merged_data["Funding Amount"].isna().sum())  # Count NaN values
merged_data = merged_data.dropna(subset=["Funding Amount"])


In [ ]:
print(merged_data["Funding Amount"].isna().sum())

In [ ]:
merged_data.info()

In [ ]:
# Making a column to capture the years of Operation in Float type

merged_data['Years of Operation'].unique()

In [ ]:
# Extracting the integer value of the number of years of the operations of the company ane making a new column out of it.

pattern = r'(?:(\d+) )?(\d4)−(\d4))?(\d{4})-(\d{4})?'

# Extract years
merged_data[["Years_Extracted", "Start_Year", "End_Year"]] = merged_data["Years of Operation"].str.extract(pattern)

# Convert Start_Year and End_Year to integers safely
merged_data["Start_Year"] = pd.to_numeric(merged_data["Start_Year"], errors="coerce").astype("Int64")
merged_data["End_Year"] = pd.to_numeric(merged_data["End_Year"], errors="coerce").astype("Int64")
merged_data["Years_Extracted"] = pd.to_numeric(merged_data["Years_Extracted"], errors="coerce")  # Keep as float

# Compute years of operation
merged_data["Years of Operation (Int)"] = merged_data["Years_Extracted"].fillna(
    merged_data["End_Year"].sub(merged_data["Start_Year"])
).astype("Int64")

# Drop intermediate columns
merged_data.drop(columns=["Years_Extracted", "Start_Year", "End_Year"], inplace=True)

# Check final result
merged_data.head()

In [ ]:
# check for NaN values in the new column
merged_data["Years of Operation (Int)"].isna().sum()

In [ ]:
# drop the unnecessary columns
merged_data.drop(columns=['Years of Operation', 'How Much They Raised'], inplace=True)

In [ ]:
merged_data.head()

In [ ]:
# check for unique values of sector
merged_data["Sector"].value_counts()

In [ ]:
# Filling the Null Values

merged_data.isnull().sum()

In [ ]:
# Dealing with null values in Platform Dependency

merged_data["Platform Dependency"] = merged_data["Platform Dependency"].fillna(0)
merged_data["Toxicity/Trust Issues"] = merged_data["Toxicity/Trust Issues"].fillna(0)
merged_data["Regulatory Pressure"] = merged_data["Regulatory Pressure"].fillna(0)
merged_data["Overhype"] = merged_data["Overhype"].fillna(0)
merged_data["High Operational Costs"] = merged_data["High Operational Costs"].fillna(0)


We choose "0" as the ideal null value for all the null instances. This is because "1" indicates the respective column was the causality of the failure of the business and "0" conveys that the column was NOT. and Since not being the causality is same as not knowing the causality we choose "0" as the ideal fill value for nulls. 

In [ ]:
merged_data.isna().sum()
# ALl the null values have been dealt with now

# Lets Start Exploring the Data

In [ ]:
# Finding a Correlation between Funding Amount, Monetization Failure and No budget

corr_matrix = merged_data[['Funding Amount', 'Monetization Failure', 'No Budget']].corr()

corr_matrix

No correlation was observed between "Funding amount" and "Monetization Failure" and "No Budget".

## Q2. What are the Top 5 Reasons for Startup Failure? (Across Industries)

In [87]:
# Let's sum up all the columns of causality and check which is the highest reasons for startup failure.

merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408 entries, 0 to 408
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name                      408 non-null    object 
 1   Sector                    408 non-null    object 
 2   What They Did             408 non-null    object 
 3   Why They Failed           408 non-null    object 
 4   Takeaway                  408 non-null    object 
 5   Giants                    408 non-null    int64  
 6   No Budget                 408 non-null    int64  
 7   Competition               408 non-null    int64  
 8   Poor Market Fit           408 non-null    int64  
 9   Acquisition Stagnation    408 non-null    int64  
 10  Platform Dependency       408 non-null    float64
 11  Monetization Failure      408 non-null    int64  
 12  Niche Limits              408 non-null    int64  
 13  Execution Flaws           408 non-null    int64  
 14  Trend Shifts   

In [103]:
import plotly.express as px

# Count occurrences of 1 in each column and sort in descending order
one_counts = merged_data[cols].apply(lambda col: (col == 1).sum()).sort_values(ascending=False)

# Create a bar chart using Plotly
fig = px.bar(
    x=one_counts.index, 
    y=one_counts.values, 
    labels={"x": "Factors", "y": "Frequency of 1s"},
    title="Frequency of '1's in Each Column",
    text=one_counts.values  # Show values on bars
)

# Customize the layout
fig.update_traces(marker_color="skyblue", textposition="outside")  # Set bar color and text position
fig.update_layout(
    xaxis_tickangle=-45,  # Rotate x-axis labels
    xaxis_title="Factors",
    yaxis_title="Frequency of 1s",
    showlegend=False,
    template="plotly_white"  # Use a clean theme
)

# Show the plot
fig.show(renderer="iframe")


In [98]:
one_counts

Giants                    308
Competition               291
Acquisition Stagnation     91
Poor Market Fit            80
No Budget                  76
Niche Limits               69
Execution Flaws            60
Monetization Failure       56
Trend Shifts               44
Toxicity/Trust Issues      23
Overhype                   19
Regulatory Pressure        14
High Operational Costs     14
Platform Dependency         7
dtype: int64

We can see that the top 5 Reasons for Startup Failure are:

1. Giants: When lost to tech giants.
2. Competition: When lost in competition.
3. Acquisition Stagnation: stagnated or faded after acquisition.
4. Poor Market Fit: if the product lacked demand or user interest.
5. No Budget: if ran out of cash or was underfunded.

ith 'Giants' and 'Competition' contributing to over 70% of the reported failure cases, their impact is nearly three times greater than the next major factor.
The first two factors—'Giants' (308 occurrences) and 'Competition' (291 occurrences)—are overwhelmingly more dominant than the third, 'Acquisition Stagnation' (91 occurrences). This indicates that market dynamics, such as established industry leaders and intense competition, play a significantly larger role in startup failures than the inability to secure acquisitions. The sheer gap in frequency suggests that startups struggle more with surviving against larger competitors and market saturation than with scaling through buyouts.

In [105]:
import pandas as pd
import plotly.express as px

# List of failure reason columns
failure_reasons = [
    "Giants", "No Budget", "Competition", "Poor Market Fit", "Acquisition Stagnation",
    "Platform Dependency", "Monetization Failure", "Niche Limits", "Execution Flaws",
    "Trend Shifts", "Toxicity/Trust Issues", "Regulatory Pressure", "Overhype", "High Operational Costs"
]

# Aggregate failure counts per sector
sector_wise_failures = merged_data.groupby("Sector")[failure_reasons].sum()

# Identify the top 5 failure reasons across all sectors
top_5_reasons = sector_wise_failures.sum().nlargest(5).index

# Filter data to keep only these reasons
sector_wise_top5 = sector_wise_failures[top_5_reasons]

# Sort sectors by total failures in descending order
sector_wise_top5["Total Failures"] = sector_wise_top5.sum(axis=1)  # Compute total failures per sector
sector_wise_top5 = sector_wise_top5.sort_values("Total Failures", ascending=False).drop(columns=["Total Failures"])

# Melt the DataFrame for Plotly
sector_wise_melted = sector_wise_top5.reset_index().melt(id_vars=["Sector"], var_name="Failure Reason", value_name="Count")

# Plot using Plotly
fig = px.bar(
    sector_wise_melted,
    x="Sector",
    y="Count",
    color="Failure Reason",
    barmode="group",  # Use "stack" for stacked bars
    title="Top 5 Reasons for Startup Failures by Sector (Sorted by Total Failures)",
    labels={"Count": "Number of Failures", "Sector": "Startup Sector"},
)

fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Failure Reason",
    bargap=0.2
)

# Show the plot
fig.show()


<ipython-input-105-390423e810eb>:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

